In [2]:
%matplotlib inline


# Classifier comparison


A comparison of a several classifiers in scikit-learn.



In [6]:
print(__doc__)


# Code source: Gaël Varoquaux
#              Andreas Müller
# Modified for documentation by Jaques Grobler
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
#from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
#from sklearn.gaussian_process import GaussianProcessClassifier
#from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

Automatically created module for IPython interactive environment


### Loading and pre-processing data

To begin with, I replaced NA's with zeros. 

Later, I will see how many features were mostly "NA". If they are not important, I will probably drop them. 

In [ ]:

trainX = [] 
with open("/home/lu/Documents/School/81_machine_learning/HW3/HW3/trainingData.txt", 'r') as rp: 
    for line in rp.read().split("\n"): 
        row = line.split("\t") 
        i = 0 
        for cell in row: 
            try: 
                row[i] = float(cell)
            except ValueError: 
                row[i] = 0 
            i+=1 
        trainX.append(row)
        
trainY = [] 
with open("/home/lu/Documents/School/81_machine_learning/HW3/HW3/trainingTruth.txt", 'r') as rp: 
    for line in rp.read().split("\n"): 
        trainY.append(line)
        
testX = [] 
with open("/home/lu/Documents/School/81_machine_learning/HW3/HW3/testData.txt", 'r') as rp: 
    for line in rp.read().split("\n"): 
        row = line.split("\t") 
        i = 0 
        for cell in row: 
            try: 
                row[i] = float(cell)
            except ValueError: 
                row[i] = 0 
            i+=1 
        testX.append(row)

In [ ]:
### Split data into training and tests sets 
arrayX = np.array(trainX[0:17377])
arrayY = np.array(trainY[0:17377])

X_train, X_test, y_train, y_test = train_test_split(arrayX, arrayY, test_size=0.4, random_state=0)



In [ ]:

classifiers = {
    "Nearest Neighbors": KNeighborsClassifier(3),
    "Linear SVC": SVC(kernel="linear", C=0.5, probability=True),
    "RBF SVM": SVC(gamma=2, C=1, probability=True),
    "DecisionTree_old": DecisionTreeClassifier(max_depth=5), 
    "RandomForest_old": RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    "DecisionTree": DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_leaf=20, random_state=0),
    "Rand Forest": RandomForestClassifier(n_estimators=30, criterion='entropy', max_depth=20, min_samples_leaf=20, 
                           bootstrap=True, oob_score=False, random_state=0 ),
    "Adaboost": AdaBoostClassifier(),
    "GaussianNB": GaussianNB(),
    "Logit": LogisticRegression(C=1), 
    "Logit l=2": LogisticRegression(C=0.5), 
    "Logit l=4": LogisticRegression(C=0.25),
    "QDA": QuadraticDiscriminantAnalysis()
}
print("Classifier | Score on Test | Score on Train ")
print("--- | --- | --- | ")


import operator
for name, classifier in  classifiers.items() : 
    clf=classifier
    clf.fit(X_train, y_train)

    ## Sanity check by predicting what is in the trainingTruth.txt file 
    prob_array = clf.predict_proba(arrayX[0:10])
    row = [
        clf.score(X_test, y_test) ,clf.score(X_train, y_train), #comparing test and training on 4/6 split
         ]
    print( "%s |"%name + "|".join(["%.3f" % x for x in row]) )
    
    for row in prob_array: 
        index, value = max(enumerate(row), key=operator.itemgetter(1))
        print(["%.3f" % x for x in row] + [index+1] )
    

Classifier | Score on Test | Score on Train 
--- | --- | --- | 
DecisionTree |0.465|0.512
['0.137', '0.155', '0.606', '0.102', 3]
['0.065', '0.298', '0.394', '0.242', 3]
['0.093', '0.250', '0.520', '0.138', 3]
['0.188', '0.400', '0.203', '0.209', 2]
['0.179', '0.307', '0.368', '0.146', 3]
['0.270', '0.392', '0.032', '0.307', 2]
['0.137', '0.155', '0.606', '0.102', 3]
['0.137', '0.155', '0.606', '0.102', 3]
['0.188', '0.400', '0.203', '0.209', 2]
['0.313', '0.238', '0.327', '0.122', 3]
QDA |0.659|1.000
['0.000', '0.000', '1.000', '0.000', 3]
['0.000', '0.000', '1.000', '0.000', 3]
['0.000', '0.000', '1.000', '0.000', 3]
['1.000', '0.000', '0.000', '0.000', 1]
['0.000', '0.000', '1.000', '0.000', 3]
['1.000', '0.000', '0.000', '0.000', 1]
['0.000', '0.000', '0.000', '1.000', 4]
['0.111', '0.889', '0.000', '0.000', 2]
['0.001', '0.999', '0.000', '0.000', 2]
['0.000', '0.000', '1.000', '0.000', 3]
Rand Forest |0.606|0.772
['0.276', '0.200', '0.415', '0.109', 3]
['0.143', '0.219', '0.507', 

### Predict using the best classifier we currently have

From the classifier comparison notebook, the best model we currently have is the GaussianNaiveBayes. Let's create and train this classifier and then use it to predict probabilities and a label per observation in the testData.txt file. 

In [44]:
clf=GaussianNB()
clf.fit(X_train, y_train)

prob_array = clf.predict_proba(testX[0:17377])

with open('predictions.csv', 'w') as wp: 
    for row in prob_array: 
        index, value = max(enumerate(row), key=operator.itemgetter(1))
        line_list = ["%.3f" % x for x in row] + [index+1] 
        line_str = '\t'.join(line_list)+'\n'
        wp.write(line_str)

/home/lu/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: setting an array element with a sequence.